In [1]:
import twitter

In [ ]:
#replaced my keys with those in the lab to show as example
api = twitter.Api(consumer_key='VXzRzW62biX8KW7A4XycqIeCL',
                      consumer_secret='Dr1ak1sdfL2CdpGCp2IWYg3xbOYFWmJ2H3Tm6ZkgMPo5ejqBrY',
                      access_token_key='78477561-2SitfsaoG4zvrq5jk1oMGahSgtBvQ9b7noe1XNNSX',
                      access_token_secret='vYNqoq4IrLpRINUQdn06aAWaeoSz7G3PNNSSt23XIlx1F')

In [3]:
print(api.VerifyCredentials())

{"created_at": "Tue Apr 24 00:21:24 +0000 2018", "default_profile": true, "default_profile_image": true, "friends_count": 1, "id": 988573587643682817, "lang": "en", "name": "Salil Harsulkar", "profile_background_color": "F5F8FA", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_link_color": "1DA1F2", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "screen_name": "SalilHarsulkar"}


In [9]:
import json
i=0
f = open('./streamingData.json', 'w')
for tweet in api.GetStreamFilter(locations=['-122.75','36.8','-121.75','37.8']):
    #print(tweet)
    f.write(json.dumps(tweet))
    f.write('\n')    
    i+=1
    if i==20:
        break
f.close()

In [10]:
with open('./streamingData.json','r') as json_data:
    d=json_data.readlines()


In [11]:
json_d=json.loads(d[1])
json_d.keys()

dict_keys(['created_at', 'id', 'id_str', 'text', 'display_text_range', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'extended_tweet', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'possibly_sensitive', 'filter_level', 'lang', 'timestamp_ms'])

In [12]:
tweets=[]
for i in range(len(d)):
    print('text for tweet-',i,':',json.loads(d[i])['text'])
    tweets.append(json.loads(d[i])['text'])

text for tweet- 0 : Growing up, I saw personally how awful these types of companies can be on low income families just trying to live paycheck to paycheck.
text for tweet- 1 : Just listed! Adorable 1920's home w/gorgeous gardens! Walk to Balboa Park, BART, Muni! https://t.co/vYe6SNj4CG $995… https://t.co/VgBY1Y1PlZ
text for tweet- 2 : I was a good girl at the phlebotomist! https://t.co/0mWu0FGVzl
text for tweet- 3 : @vyrus001 You need to unlock your Twitter account so I can retweet your post! 😂
text for tweet- 4 : @SheaSerrano literally did the same thing... I’m ready https://t.co/LZMwOltEwT
text for tweet- 5 : @M_G_abdelhameed هتتسد من التراب!
text for tweet- 6 : @stephneehope Blows I know 😔
text for tweet- 7 : went to orange theory yesterday + everything hurts + I’m dying
text for tweet- 8 : My motivation for school is no where to be found 😒
text for tweet- 9 : @Friendstagram I am.
text for tweet- 10 : @jeffayc You seen Schrader's Dog Eat Dog? He plays the straight man to Defoe's lun

#### Sentiment analysis -
https://nlpforhackers.io/sentiment-analysis-intro/

In [89]:
#import required libraries
import re
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn 
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
import numpy as np

In [37]:
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\hsalil89\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [15]:
stop = stopwords.words('english')

In [20]:
replacement_dict={"http\S+":"","@\S+":"","[\n]":"","'m":" am","’ve":" have","in’":"ing","’s":" is","w/":" with ","[^a-zA-Z\s]+":"" }

In [21]:
patterns=replacement_dict.keys()

#### Sentiment Analysis pipeline -
In this pipeline we perform the below steps -
    1. Clean tweet and remove non-characters using Regex
    2. Tokenize the tweets to seperate words
    3. Remove stop words
    4. Remove duplicate words
    5. Extract POS tags for each word and convert tags to wordnet format
    6. For each word, lemmatize to remove plurals 
    7. Identify the sentiment and consolidate it for each tweet 

In [90]:
parsed_tweets=[]
ptb_wn_pos_dict= { 'NN' : 'n', 'JJ' : 'a', 'RB' : 'a', 'VB' : 'v' }
lemmatizer = WordNetLemmatizer()
for tweet in tweets:
    sentiment = 0.0
    tokens_count = 0
    for pattern in patterns:
        tweet = re.sub(pattern, lambda m: replacement_dict.get(m.group()), tweet)
    #print(tweet)
    parsed_tweets.append(tweet)
    #tokenize
    tweet_token= nltk.tokenize.word_tokenize(tweet)
    #remove stop words
    new_tweet=[str(word).lower() for word in tweet_token if str(word).lower() not in stop]
    #print(new_tweet)
    tweet_tagged=nltk.pos_tag(np.unique(new_tweet))
    for word,tag in tweet_tagged:
        if ptb_wn_pos_dict.get(tag,'d') in ['n','a','v']:
            pos_wn=ptb_wn_pos_dict[tag]
            #lemmatize
            lemma=lemmatizer.lemmatize(word, pos_wn) 
            #sentiment extraction
            synsets = wn.synsets(lemma, pos=pos_wn)
            if not synsets:
                continue
            #print(lemma,pos_wn) 
            # Take the first sense, the most common
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
 
            sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            tokens_count += 1
    print('Tweet: ',tweet)
    print('Sentiment score = ',sentiment)
    
    #POS tageach word
    #pos_score.append(np.sum([]))

Tweet:  Growing up I saw personally how awful these types of companies can be on low income families just trying to live paycheck to paycheck
Sentiment score =  -1.125
Tweet:  Just listed Adorable s home  with gorgeous gardens Walk to Balboa Park BART Muni   
Sentiment score =  1.25
Tweet:  I was a good girl at the phlebotomist 
Sentiment score =  0.75
Tweet:   You need to unlock your Twitter account so I can retweet your post 
Sentiment score =  0.0
Tweet:   literally did the same thing Im ready 
Sentiment score =  0.0
Tweet:     
Sentiment score =  0.0
Tweet:   Blows I know 
Sentiment score =  0.0
Tweet:  went to orange theory yesterday  everything hurts  Im dying
Sentiment score =  0.0
Tweet:  My motivation for school is no where to be found 
Sentiment score =  0.0
Tweet:   I am
Sentiment score =  0.0
Tweet:   You seen Schraders Dog Eat Dog He plays the straight man to Defoes lunatic Like it or not it is art
Sentiment score =  -0.875
Tweet:  everything is going to be okay everything